# Correcting songs and fine-tuing

## Import modules

In [1]:
import os
from neuspell import available_checkers, BertChecker, SclstmChecker

print(f"available checkers: {available_checkers()}")

data folder is set to `e:\nlp\nlp-env\neuspell\neuspell\../data` script
available checkers: ['BertsclstmChecker', 'CnnlstmChecker', 'ElmosclstmChecker', 'NestedlstmChecker', 'SclstmChecker', 'SclstmbertChecker', 'SclstmelmoChecker', 'BertChecker']


## Define data path

In [2]:
train_data_path = '../dataset/train'
clean_train_path = os.path.join(train_data_path, 'clean_lyrics.txt')
noisy_train_path = os.path.join(train_data_path, 'noisy_lyrics.txt')

test_data_path = '../dataset/test'
clean_test_path = os.path.join(test_data_path, 'test_lyrics.txt')
noisy_test_path = os.path.join(test_data_path, 'test_lyrics_noisy.txt')

## Load training data

In [3]:
with open(clean_train_path) as f:
    clean_train = f.read().splitlines()

clean_train[:10]

['just a lost boy in a small town',
 'singing "love is forever and ever"',
 'good on paper, picture perfect',
 "cause i know i'm addicted to your drama",
 'you see me i be',
 'i gotta tell them to myself',
 "i'm still learning to love",
 "no, i can't sleep until i feel your touch",
 'and all i can think',
 "so beautiful you're leaving me"]

In [4]:
with open(noisy_train_path) as f:
    noisy_train = f.read().splitlines()

noisy_train[:10]

['just a lsot boy in a smmall tiwn',
 'singing "lve is foreiver and eve"',
 'good on paper, piccture perfect',
 "cayse i know i'm adicted to your dtama",
 'yoou see me i be',
 'i gotta tell tehm to myseclf',
 "i'm sitll learning to lovke",
 "no, i can't sleep until i feel your toch",
 'and all i caan think',
 "so beautiful yuo're letaving me"]

In [5]:
with open(clean_test_path) as f:
    clean_test = f.read().splitlines()

clean_test[:10]

['the world on drugs',
 'ten yeah thousand dollar plates',
 'thousand dollar plates',
 'thousand dollar plates, fine china',
 'shorty like a, uh',
 'shorty like a',
 'wheezy outta here',
 'shorty like a thousand dollar plate, fine china',
 'tell her that she beautiful every day, i remind her',
 "then i jump in the pussy like a lake, i'm a diver"]

In [6]:
with open(noisy_train_path) as f:
    noisy_test = f.read().splitlines()

noisy_test[:10]

['just a lsot boy in a smmall tiwn',
 'singing "lve is foreiver and eve"',
 'good on paper, piccture perfect',
 "cayse i know i'm adicted to your dtama",
 'yoou see me i be',
 'i gotta tell tehm to myseclf',
 "i'm sitll learning to lovke",
 "no, i can't sleep until i feel your toch",
 'and all i caan think',
 "so beautiful yuo're letaving me"]

## BERTChecker

### Load pretrained model

In [7]:
# create BertChecker
bert_checker = BertChecker()
# use pretrained model
bert_checker.from_pretrained()

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 185211810
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise


### Evaluate pretrained model on testing set

In [8]:
bert_checker.evaluate(clean_file="test_lyrics.txt", corrupt_file="test_lyrics_noisy.txt", data_dir=test_data_path)

../dataset/test test_lyrics.txt test_lyrics_noisy.txt


1418it [00:00, 472625.80it/s]
1418it [00:00, 708291.42it/s]


loaded tuples of (corr,incorr) examples from ../dataset/test
###############################################
data size: 1418


355it [01:29,  3.96it/s]


Epoch None valid_loss: 0.6923443188262008
total inference time for this data is: 89.627995 secs
###############################################


total token count: 11906
_corr2corr:8991, _corr2incorr:664, _incorr2corr:1761, _incorr2incorr:490
accuracy is 0.9030740802956493
word correction rate is 0.7823189693469569
###############################################


### Example of wrong outputs 

In [9]:
bert_correct = bert_checker.correct_strings(noisy_test)
bert_correct[:10]

KeyboardInterrupt: 

In [ ]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
bert_correct_prep = [i.replace(' ', '').strip().lower() for i in bert_correct]

print('len data:', len(clean_test_prep))
print('Number of correct data after bert:', sum(a == b for a, b in zip(bert_correct_prep, clean_test_prep)))

example: justalotboyinasmalltown
len data: 7309
correct data after bert: 4404


In [ ]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != bert_correct_prep[i]:
        print(clean_test[i], ' | ', bert_correct[i])

just a lost boy in a small town  |  just a lot boy in a small town
singing "love is forever and ever"  |  singing " love is forever and even "
i'm still learning to love  |  I ' m still learning to look
i feel like i could die walking up to the room, oh yeah  |  I feel like I could die waking up to the room , oh yeah
i'm really gonna miss you picking fights  |  I ' m really gonna miss you picking gifts
even tried to bite my tongue when you start shit  |  even tried to bite my tongue when you start shot
what the fuck did i do  |  what the folk did i do
blurring all the lines, you intoxicate me  |  blurring all the lines , you intoxicated me
kick in the door waving the coco  |  kick in the floor wearing the condo
and your craigslist couch and the way your voice sounds  |  and your craigslist such as the way your voice sounds
vroom vroom, i'll see you latеr, bye  |  from freedom , I ' ll see you later , bye
and a neigh neigh there.  |  and a new night there .
i just wish i can be there wi

### Fine-tuning BertChecker

In [ ]:
# create new BertChecker
bert_checker_new = BertChecker()
# load pretrained model
bert_checker_new.from_pretrained()
# fine-tune on training data
bert_checker_new.finetune(clean_file="clean_lyrics.txt", corrupt_file="noisy_lyrics.txt", data_dir=train_data_path)

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 185211810
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise


7309it [00:00, 811976.37it/s]
7309it [00:00, 1461139.50it/s]


loaded tuples of (corr,incorr) examples from e:\nlp\nlp-env\neuspell\neuspell\../data\traintest
len of train and test data:  5848 1461
CHECKPOINT_PATH: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased
Training model params
In epoch: 0
train_data size: 5848
Percent: [>                             ] 1% || batch_time: 4.1719 || batch_loss: 0.7811 || avg_batch_loss: 0.5298 || batch_acc: 0.9580 || avg_batch_acc: 0.9580 

e:\NLP\nlp-env\lib\site-packages\pytorch_pretrained_bert\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Percent: [----------------------------->] 100% || batch_time: 3.0696 || batch_loss: 0.0887 || avg_batch_loss: 0.2350 || batch_acc: 0.9580 || avg_batch_acc: 0.9580 
Epoch 0 train_loss: 0.23499097294540558
valid_data size: 1461
Percent: [----------------------------->] 100% || batch_time: 1.9114 || batch_loss: 0.0270 || avg_batch_loss: 0.1347 || batch_acc: 0.9938 || avg_batch_acc: 0.9714 
Epoch 0 valid_loss: 0.1346610203061415
validation accuracy improved from -1.0000 to 44.6831
Model saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased\pytorch_model.bin in epoch 0
In epoch: 1
train_data size: 5848
Percent: [----------------------------->] 100% || batch_time: 3.6861 || batch_loss: 0.0008 || avg_batch_loss: 0.0230 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 

...

Percent: [----------------------------->] 99% || batch_time: 3.4971 || batch_loss: 0.0012 || avg_batch_loss: 0.0084 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 

Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


Percent: [----------------------------->] 100% || batch_time: 3.6199 || batch_loss: 0.0036 || avg_batch_loss: 0.0084 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 

Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.


Percent: [----------------------------->] 100% || batch_time: 2.9512 || batch_loss: 0.0012 || avg_batch_loss: 0.0084 || batch_acc: 1.0000 || avg_batch_acc: 1.0000 
Epoch 2 train_loss: 0.008352796749966957
valid_data size: 1461
Percent: [----------------------------->] 100% || batch_time: 1.9136 || batch_loss: 0.0135 || avg_batch_loss: 0.1153 || batch_acc: 0.9938 || avg_batch_acc: 0.9754 
Epoch 2 valid_loss: 0.11528032609140096
validation accuracy improved from 44.8663 to 44.8663
Model saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased\pytorch_model.bin in epoch 2
Model and logs saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\new_models\bert-base-cased


### Evaluate on testing set after fine-tuning

In [ ]:
bert_checker_new.evaluate(clean_file="test_lyrics.txt", corrupt_file="test_lyrics_noisy.txt", data_dir=test_data_path)

e:\nlp\nlp-env\neuspell\neuspell\../data\traintest clean_lyrics.txt noisy_lyrics.txt


7309it [00:00, 730743.90it/s]
7309it [00:00, 1217577.57it/s]


loaded tuples of (corr,incorr) examples from e:\nlp\nlp-env\neuspell\neuspell\../data\traintest
###############################################
data size: 7309


1828it [07:04,  4.31it/s]


Epoch None valid_loss: 0.026905515569071756
total inference time for this data is: 424.256846 secs
###############################################


total token count: 58943
_corr2corr:43238, _corr2incorr:60, _incorr2corr:14843, _incorr2incorr:802
accuracy is 0.9853757019493409
word correction rate is 0.9487376158517098
###############################################


### Result analysis

In [ ]:
bert_correct_new = bert_checker_new.correct_strings(noisy_test)
bert_correct_new[:10]

['just a lost boy in a small town',
 'singing " love is forever and eve "',
 'good on paper , picture perfect',
 "cause i know i ' m addicted to your drama",
 'you see me i be',
 'i gotta tell them to myself',
 "i ' m still learning to love",
 "no , i can ' t sleep until i feel your touch",
 'and all i can think',
 "so beautiful you ' re leaving me"]

In [ ]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
bert_correct_new_prep = [i.replace(' ', '').strip().lower() for i in bert_correct_new]

print('len data:', len(clean_test_prep))
print('correct data after bert:', sum(a == b for a, b in zip(bert_correct_new_prep, clean_test_prep)))

example: justalostboyinasmalltown
len data: 7309
correct data after bert: 6456


In [ ]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != bert_correct_new_prep[i]:
        print(clean_test_prep[i], ' | ', bert_correct_new[i])

singing "love is forever and ever"  |  singing " love is forever and eve "
i'm really gonna miss you picking fights  |  i ' m really gonna miss you picking gifts
blurring all the lines, you intoxicate me  |  blurring all the lines , you inotxicate me
kick in the door waving the coco  |  kick in the door waving the codo
vroom vroom, i'll see you latеr, bye  |  from vriom , i ' ll see you later , bye
and a neigh neigh there.  |  and a neih neigh there .
i gave a second chance to cupid  |  i gave a second chance to cuepid
that'll work, come over  |  thrill work , come over
got a lot of love, well you better save it for me  |  got a lot of love , well you better saved it for me
we went to bed in france, then we woke up in japan  |  we went to bed in france , then we woke up in jaoan
they don't want to see us get too attached  |  they dont want to see us get too attached
oh, make you try fo understand  |  oh , make you try to understand
oh, i might stop talking to people before i snap, snap

## SC-LSTM plus ELMO (at input)

### Load pretrained model

In [ ]:
# create SclstmChecker
elmo_checker = SclstmChecker()
# add ELMO at input
elmo_checker = elmo_checker.add_("elmo", at="input")  # "elmo" or "bert", "input" or "output"
# load pretrained model
elmo_checker.from_pretrained()

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise
new model loaded: <class 'neuspell.corrector_elmosclstm.ElmosclstmChecker'>
loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise
creating spacy models ...
spacy models initialized


e:\NLP\nlp-env\lib\site-packages\torch\nn\modules\container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


['just a lost boy in a small town',
 'singing " love is forever and ever "',
 'good on paper , picture perfect',
 "cause I know i 'm addicted to your data",
 'you see me and be',
 'I got to tell them to myself',
 "I 'm still learning to love",
 'no , I can not sleep until i feel your touch',
 'and all I can think',
 'so beautiful your leaving me']

### Evaluate pretrained model on testing set

In [ ]:
elmo_checker.evaluate(clean_file="test_lyrics.txt", corrupt_file="test_lyrics_noisy.txt", data_dir=test_data_path)

e:\nlp\nlp-env\neuspell\neuspell\../data\traintest clean_lyrics.txt noisy_lyrics.txt


7309it [00:00, 812126.95it/s]
7309it [00:00, 1217916.17it/s]


loaded tuples of (corr,incorr) examples from e:\nlp\nlp-env\neuspell\neuspell\../data\traintest
data size: 7309


1828it [14:40,  2.08it/s]


Epoch None valid_loss: 1.237444352181441
total inference time for this data is: 880.299118 secs
###############################################
total token count: 49947
corr2corr:30071, corr2incorr:4235, incorr2corr:10823, incorr2incorr:4818
accuracy is 0.8187478727451099
word correction rate is 0.691963429448245
###############################################


### Example of wrong outputs

In [ ]:
# correct noisy data
elmo_correct = elmo_checker.correct_strings(noisy_test)
elmo_correct[:10]

In [ ]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
elmo_correct_prep = [i.replace(' ', '').strip().lower() for i in elmo_correct]

print('len data:', len(clean_test_prep))
print('correct data after bert:', sum(a == b for a, b in zip(elmo_correct_prep, clean_data_prep)))

example: justalostboyinasmalltown
len data: 7309
correct data after bert: 3885


In [ ]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != elmo_correct_prep[i]:
        print(clean_test[i], ' | ', elmo_correct[i])

cause i know i'm addicted to your drama  |  cause I know i 'm addicted to your data
you see me i be  |  you see me and be
i gotta tell them to myself  |  I got to tell them to myself
no, i can't sleep until i feel your touch  |  no , I can not sleep until i feel your touch
so beautiful you're leaving me  |  so beautiful your leaving me
when it wasn't yours, yeah  |  when it was not yours , yeah
i feel like i could die walking up to the room, oh yeah  |  I feel like I could die waking up to the room , oh yeah
i'm really gonna miss you picking fights  |  I 'm really on no misos you picking fights
even tried to bite my tongue when you start shit  |  even tried to bite my tongue when you start shut
and i won't be your victim  |  and it wont be your victim
what the fuck did i do  |  what the fuck did it do
blurring all the lines, you intoxicate me  |  blurring all the lines , you inotxicate me
kick in the door waving the coco  |  kick in the door waving the code
and your craigslist couch an

### Fine-tuning SclstmChecker

In [ ]:
# create AclstmChecker
elmo_checker_new = SclstmChecker()
# add elmo at input
elmo_checker_new = elmo_checker_new.add_("elmo", at="input")  # "elmo" or "bert", "input" or "output"
# load pretrained model
elmo_checker_new.from_pretrained()
# fine-tune on training data
elmo_checker_new.finetune(clean_file="clean_lyrics.txt", corrupt_file="noisy_lyrics.txt", data_dir=train_data_path)

loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise
new model loaded: <class 'neuspell.corrector_elmosclstm.ElmosclstmChecker'>
loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 209906438
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise


7309it [00:00, 730709.06it/s]
7309it [00:00, 1826729.11it/s]


loaded tuples of (corr,incorr) examples from e:\nlp\nlp-env\neuspell\neuspell\../data\traintest
len of train and test data:  5848 1461
CHECKPOINT_PATH: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\new_models\elmo-base-cased
Training model params from scratch
In epoch: 0
train_data size: 5848
Percent: [----------------------------->] 100% || batch_time: 1.7134 || batch_loss: 0.0763 || avg_batch_loss: 0.2813 || batch_acc: 0.7664 || avg_batch_acc: 0.7664 
Epoch 0 train_loss: 0.2813334097383452
valid_data size: 1461
Percent: [----------------------------->] 100% || batch_time: 1.3783 || batch_loss: 0.0173 || avg_batch_loss: 0.1022 || batch_acc: 1.0000 || avg_batch_acc: 0.9698 
Epoch 0 valid_loss: 0.10223011644152195
Model saved at e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/elmoscrnn-probwordnoise\new_models\elmo-base-cased\model.pth.tar in epoch 0
In epoch: 1
train_data size: 5848
Percent: [----------------------------->] 100% || batch_time: 1.5775

### Evaluate on testing set after fine-tuning

In [ ]:
elmo_checker_new.evaluate(clean_file="test_lyrics.txt", corrupt_file="test_lyrics_noisy.txt", data_dir=test_data_path)

e:\nlp\nlp-env\neuspell\neuspell\../data\traintest clean_lyrics.txt noisy_lyrics.txt


7309it [00:00, 811976.37it/s]
7309it [00:00, 1044076.29it/s]


loaded tuples of (corr,incorr) examples from e:\nlp\nlp-env\neuspell\neuspell\../data\traintest
data size: 7309


1828it [14:29,  2.10it/s]


Epoch None valid_loss: 0.024568097715663473
total inference time for this data is: 869.061731 secs
###############################################
total token count: 49947
corr2corr:34261, corr2incorr:45, incorr2corr:11927, incorr2incorr:3714
accuracy is 0.9247402246381164
word correction rate is 0.7625471517166421
###############################################


### Result analysis

In [ ]:
elmo_correct_new = elmo_checker_new.correct_strings(noisy_test)
elmo_correct_new[:10]

['just a lost boy in a small town',
 'singing " love is forever and eve "',
 'good on paper , picture perfect',
 "cause i know i 'm addicted to your drama",
 'you see me i be',
 'i got to tell them to myself',
 "i 'm still learning to love",
 'no , i can not sleep until i feel your touch',
 'and all i can think',
 "so beautiful yuo're leaving me"]

In [ ]:
clean_test_prep = [i.replace(' ', '').strip().lower() for i in clean_test]
elmo_correct_new_prep = [i.replace(' ', '').strip().lower() for i in elmo_correct_new]

print('len data:', len(clean_test_prep))
print('correct data after bert:', sum(a == b for a, b in zip(elmo_correct_new_prep, clean_test_prep)))

example: justalostboyinasmalltown
len data: 7309
correct data after bert: 4988


In [ ]:
for i in range(len(clean_test_prep)):
    if clean_test_prep[i] != elmo_correct_new_prep[i]:
        print(clean_test[i], ' | ', elmo_correct_new[i])

singing "love is forever and ever"  |  singing " love is forever and eve "
i gotta tell them to myself  |  i got to tell them to myself
no, i can't sleep until i feel your touch  |  no , i can not sleep until i feel your touch
so beautiful you're leaving me  |  so beautiful yuo're leaving me
when it wasn't yours, yeah  |  when it was not yours , yeah
and i won't be your victim  |  and i wn't be your victim
blurring all the lines, you intoxicate me  |  blurring all the lines , you inotxicate me
kick in the door waving the coco  |  kick in the door waving the codo
vroom vroom, i'll see you latеr, bye  |  vfoom vriom , i 'll see you later , bye
and a neigh neigh there.  |  and a neih neigh there .
i can't describe  |  i cn't describe
if you love me won't you  |  if you love me wons't you
don't leave me stuck here in the streets no, no  |  don'xt leave me stuck here in the streets no , no
cause i'm in a field of dandelions  |  cause i'lm in a field of dandelions
i don't mean no harm  |  i 